# Visualize 3D Time-Series of Meshes

This notebook does not run on the server: run on your local machine.

## Set-up and Imports

In [1]:
import os
import subprocess

gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
print("Working directory: ", os.getcwd())

import warnings

warnings.filterwarnings("ignore")

import sys

sys_dir = os.path.dirname(os.getcwd())
sys.path.append(sys_dir)

Working directory:  /Volumes/GoogleDrive/My Drive/code/my28brains


In [2]:
import glob
import os
from time import sleep

import numpy as np
import open3d as o3d
import trimesh

import my28brains.viz as viz

REGISTERED_MESHES_DIR = os.path.join(os.getcwd(), "data", "registered_meshes")
print("REGISTERED_MESHES_DIR: ", REGISTERED_MESHES_DIR)

REGISTERED_MESHES_DIR:  /Volumes/GoogleDrive/My Drive/code/my28brains/data/registered_meshes


In [3]:
hemisphere = "left"
ply_suffix = f"ashs/{hemisphere}_lfseg_corr_usegray_CT_LQ.ply"
string_base = os.path.join(REGISTERED_MESHES_DIR, f"sub-01/ses-**/{ply_suffix}")
paths = sorted(glob.glob(string_base))

print(f"Found {len(paths)} ply files.")
print(paths)

Found 27 ply files.
['/Volumes/GoogleDrive/My Drive/code/my28brains/data/registered_meshes/sub-01/ses-02/ashs/left_lfseg_corr_usegray_CT_LQ.ply', '/Volumes/GoogleDrive/My Drive/code/my28brains/data/registered_meshes/sub-01/ses-03/ashs/left_lfseg_corr_usegray_CT_LQ.ply', '/Volumes/GoogleDrive/My Drive/code/my28brains/data/registered_meshes/sub-01/ses-04/ashs/left_lfseg_corr_usegray_CT_LQ.ply', '/Volumes/GoogleDrive/My Drive/code/my28brains/data/registered_meshes/sub-01/ses-05/ashs/left_lfseg_corr_usegray_CT_LQ.ply', '/Volumes/GoogleDrive/My Drive/code/my28brains/data/registered_meshes/sub-01/ses-06/ashs/left_lfseg_corr_usegray_CT_LQ.ply', '/Volumes/GoogleDrive/My Drive/code/my28brains/data/registered_meshes/sub-01/ses-07/ashs/left_lfseg_corr_usegray_CT_LQ.ply', '/Volumes/GoogleDrive/My Drive/code/my28brains/data/registered_meshes/sub-01/ses-08/ashs/left_lfseg_corr_usegray_CT_LQ.ply', '/Volumes/GoogleDrive/My Drive/code/my28brains/data/registered_meshes/sub-01/ses-09/ashs/left_lfseg_corr

## Visualize one Mesh

In [4]:
mesh = o3d.io.read_triangle_mesh(
    os.path.join(REGISTERED_MESHES_DIR, f"sub-01/ses-03/{ply_suffix}")
)
mesh.compute_vertex_normals()
mesh.paint_uniform_color([1, 0.706, 0])

o3d.visualization.draw_geometries([mesh])

## Create an animation with point clouds

In [5]:
video_dir = os.path.join(os.getcwd(), "data", "video")
if not os.path.exists(video_dir):
    os.makedirs(video_dir)

for i_path, path in enumerate(paths):  ## TODO REPLACE
    mesh = trimesh.load(path)
    ply_text = trimesh.exchange.ply.export_ply(mesh, encoding="ascii")

    ply_path = os.path.join(video_dir, f"left_brain{i_path}.ply")
    print(f"Writing mesh at {ply_path}...")
    with open(ply_path, "wb") as f:
        f.write(ply_text)

Writing mesh at /Volumes/GoogleDrive/My Drive/code/my28brains/data/video/left_brain0.ply...
Writing mesh at /Volumes/GoogleDrive/My Drive/code/my28brains/data/video/left_brain1.ply...
Writing mesh at /Volumes/GoogleDrive/My Drive/code/my28brains/data/video/left_brain2.ply...
Writing mesh at /Volumes/GoogleDrive/My Drive/code/my28brains/data/video/left_brain3.ply...
Writing mesh at /Volumes/GoogleDrive/My Drive/code/my28brains/data/video/left_brain4.ply...
Writing mesh at /Volumes/GoogleDrive/My Drive/code/my28brains/data/video/left_brain5.ply...
Writing mesh at /Volumes/GoogleDrive/My Drive/code/my28brains/data/video/left_brain6.ply...
Writing mesh at /Volumes/GoogleDrive/My Drive/code/my28brains/data/video/left_brain7.ply...
Writing mesh at /Volumes/GoogleDrive/My Drive/code/my28brains/data/video/left_brain8.ply...
Writing mesh at /Volumes/GoogleDrive/My Drive/code/my28brains/data/video/left_brain9.ply...
Writing mesh at /Volumes/GoogleDrive/My Drive/code/my28brains/data/video/left_br

In [6]:
video_dir = os.path.join(os.getcwd(), "data", "video")
if not os.path.exists(video_dir):
    os.makedirs(video_dir)

vis = o3d.visualization.Visualizer()
vis.create_window()

pcd = o3d.io.read_point_cloud(os.path.join(video_dir, "left_brain0.ply"))
vis.add_geometry(pcd)
vis.poll_events()
vis.update_renderer()

for i in range(1, len(paths)):
    sleep(1)
    pcd.points = o3d.io.read_point_cloud(
        os.path.join(video_dir, f"left_brain{i}.ply")
    ).points
    vis.update_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()